### Import of Toolboxes and Input Data

In [1]:
import scipy.io as spio
import numpy as np
import pandas as pd
import pixiedust
import inspect
import matplotlib.pyplot as plt
%matplotlib notebook

Pixiedust database opened successfully


In [2]:
mat = spio.loadmat('./data/WorstOffAutocallablePayoff_inputs.mat', squeeze_me=True)

### Defininition of the Payoff Function

In [3]:
def worst_off_autocallable_payoff(para_prod, risk_factor, barrier_factor,num_coupon_fix,num_sim):
    import numpy as np

    # Index Calculation
    # 'rf1' is obtained from the 3-d tensor 'risk_factor' 
    # by using all its slices except the first one (risk_factor[:,:,[0]]) 
    # used to renormalize the others
    rf1 = np.divide(risk_factor[:,:,1:],risk_factor[:,:,[0]])
    
    # Correction for Worst of
    # 'rf' is a 2-d array obtained by taking the minimum along the second axis of 'rf1'
    # 'rf1' is a 3-d tensor where each dimension represents respectively 
    # the number of simulations, the assets and the number of events.
    rf = np.amin(rf1,axis=1)
    # 'bf1' is obtained by normalizing the 2-d array 'barrier_factor' using the first slice of 'risk_factor'    
    bf1 = barrier_factor / risk_factor[:,:,0]
    # 'bf' is obtained by taking the minimum along the columns of 'bf1'
    bf = np.min(bf1, axis=1)
        
    # Digital Effect
    # 'pointer_digital_effect' and 'coupon_digital' 2-d arrays are obtained using two different portions of the para_prod array
    # 'digital_effect' 2-d array is obtained creating a boolean, which checks where 'rf' is bigger than the first 24 elements of
    # 'para_prod' 1-d array, which is then multiplied by 'coupon_digital' and 'pointer_digital_effect'
    pointer_digital_effect = para_prod[-(num_coupon_fix*3):-(num_coupon_fix*2), None].T        
    coupon_digital = para_prod[(num_coupon_fix*2):(num_coupon_fix*3)]
    digital_effect = (np.greater_equal(rf, para_prod[:num_coupon_fix]) * coupon_digital ) * pointer_digital_effect
    
    # Autocall Effect
    # the 'pointer_autocall_effect' 1-d array is obtained taking a portion of 'para_prod' 1-d array
    pointer_autocall_effect = para_prod[-(num_coupon_fix*2):-(num_coupon_fix), None].T
    
    # Pointer Matrix
    # - 'pointer_auto_call' is initially obtained creating a boolean, defining where 'rf' is bigger than a specified portion of 
    #    the 'para_prod' 1-d array
    # - it is defined a list of indices indicating where the 'pointer_autocall_effect' 1-d array elements are equal to 0
    # - the 'pointer_auto_call' values are replaced with 0 using the defined list od indices, by columns
    # - it is calculcated the cumulated sum by rows of 'pointer_auto_call'
    # - the elements of 2-d array 'pointer_auto_call' bigger than 1 are replaced by 30000
    # - the elements of the last column of 'pointer_auto_call' equal to 0 are replaced by 10000
    # - the elements 'pointer_auto_call' equal to 10000 are replaced by 3
    # - the elements 'pointer_auto_call' equal to 20000 are replaced by 2
    # - the elements 'pointer_auto_call' equal to 30000 are replaced by 0
    pointer_auto_call = np.greater_equal(rf, para_prod[num_coupon_fix:num_coupon_fix*2]).astype(int)
    pointer_auto_call[:, np.where(pointer_autocall_effect == 0)] = 0
    pointer_auto_call = np.cumsum(pointer_auto_call, axis=1)
    pointer_auto_call[np.where(pointer_auto_call > 1)] = 30000
    pointer_auto_call[np.where(pointer_auto_call[:,-1] == 0)] = 10000
    pointer_auto_call[np.where(pointer_auto_call == 10000)] = 3
    pointer_auto_call[np.where(pointer_auto_call == 20000)] = 2
    pointer_auto_call[np.where(pointer_auto_call == 30000)] = 0
    
    # Autocall Coupon T-
    # - 'auto_call_coupon' is equal to 2-d 'digital_effect' array
    # - the 'auto_call_coupon' values are replaced with 1 where the 'pointer_auto_call' 2-d array elements are equal to 1
    # - the 'auto_call_coupon' values are replaced with 0 where the 'pointer_auto_call' 2-d array elements are equal to 0
    auto_call_coupon = digital_effect
    auto_call_coupon[np.where(pointer_auto_call == 1)] += 1
    auto_call_coupon[np.where(pointer_auto_call == 0)] = 0
    
    # Autocall Coupon T
    # - 'pointer_auto_call_end' is obtained taking the last column of 'pointer_auto_call' 2-d array
    # - 'payoff_end' is initially set equal to 1-d array 'bf'
    # - 'payoff_end' is set equal to a defined value of 'para_prod' for the indices in which 'bf' is grater than or equal to a 
    #    different specific value of 'para_prod'
    # - 'payoff_end' is set equal to a defined value of 'para_prod' for the indices in which 'bf' is grater than or equal to a 
    #    different specific value of 'para_prod'
    # -  the 'payoff_end' values are replaced with 0 where the 'pointer_auto_call_end' 1-d array elements are different to 3
    # -  the last column of 'auto_call_coupon' is replaced with 1-d 'payoff_end' array
    pointer_auto_call_end = pointer_auto_call[:,-1]
    payoff_end = np.copy(bf)
    payoff_end[np.greater_equal(bf,para_prod[num_coupon_fix*2-1])] = para_prod[num_coupon_fix*3]
    payoff_end[np.greater_equal(bf,para_prod[num_coupon_fix-1])] = para_prod[num_coupon_fix*3+1]
    payoff_end[np.where(pointer_auto_call_end != 3)] = 0
    auto_call_coupon[:,-1] = payoff_end
    
    # Fix Coupon
    # - 2-d array 'coupon_fix' is obtained as the repetition of a portion of 'para_prod' how many times as the number of 
    #   simulation are
    # - 'payoff' is equal to 'auto_call_coupon' 2-d array
    coupon_fix = np.repeat(para_prod[-(num_coupon_fix):][np.newaxis].T, num_sim, axis=1).T
    payoff = auto_call_coupon
    
    return payoff, coupon_fix

### Function Execution

In [4]:
#%%pixie_debugger
Payoff, CouponFix = worst_off_autocallable_payoff(mat['ParaProd'], mat["RiskFactor"], mat["BarrierFactor"], mat["NumCouponFix"], mat["NumSim"])
Payoff = pd.DataFrame(Payoff)
CouponFix = pd.DataFrame(CouponFix)

### Load Matlab Results

In [5]:
MatPayoff = pd.read_csv('data/Payoff.csv', header=None)
MatCouponFix = pd.read_csv('data/CouponFixed.csv', header=None)

### Check compatibility with matlab results

In [6]:
assert np.isclose(Payoff,MatPayoff).all()
assert np.isclose(CouponFix, MatCouponFix).all()

### Differences between Python and Matlab

In [7]:
payoff_diff = ((Payoff - MatPayoff) / MatPayoff).round(15)
payoff_diff.fillna(0, inplace=True)
coupon_fix_diff = ((CouponFix - MatCouponFix) / MatCouponFix).round(15)
coupon_fix_diff.fillna(0, inplace=True)

### Payoff Results

In [8]:
res=pd.DataFrame(np.transpose([Payoff.iloc[:,-1], MatPayoff.iloc[:,-1], payoff_diff.iloc[:,-1].abs()]), columns=["Payoff Python", "Payoff Matlab", "Differences"])

In [9]:
res

,Payoff Python,Payoff Matlab,Differences
0,0.033646,0.033646,0.0
1,0.075774,0.075774,0.0
2,0.201629,0.201629,0.0
3,0.110093,0.110093,0.0
4,1.000000,1.000000,0.0
5,1.010000,1.010000,0.0
6,1.010000,1.010000,0.0
7,0.231149,0.231149,0.0
8,0.095644,0.095644,0.0
9,1.010000,1.010000,0.0
